In [16]:
import pymc3 as pm
import numba
import numpy as np
import matplotlib as mat
import pandas as pd
import theano.tensor as tt
from scipy.special import expit 

invlogit = lambda x: 1/(1 + tt.exp(-x))

In [2]:
df = pd.read_csv('../ssl-sgs/train.csv')
y = np.asarray(df.target)
X = np.array(df.iloc[:, 2:302])

In [3]:
df2 = pd.read_csv('../ssl-sgs/test.csv')
df2.head()
X2 = np.array(df2.iloc[:, 1:301])

In [4]:
def get_model(y, X):
    model = pm.Model()
    with model:
        xi = pm.Bernoulli('xi', .05, shape=X.shape[1]) #inclusion probability for each variable
        alpha = pm.Normal('alpha', mu = 0, sd = 5) # Intercept
        beta = pm.Normal('beta', mu = 0, sd = .75 , shape=X.shape[1]) #Prior for the non-zero coefficients
        p = pm.math.dot(X, xi * beta) #Deterministic function to map the stochastics to the output
        y_obs = pm.Bernoulli('y_obs', invlogit(p + alpha),  observed=y)  #Data likelihood
    return model

In [5]:
model1 = get_model(y, X)

In [6]:
with model1:
    trace = pm.sample(2000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)

c:\Users\Johnny Lee\anaconda3\envs\py39win64\lib\site-packages\deprecat\classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
c:\Users\Johnny Lee\anaconda3\envs\py39win64\lib\site-packages\theano\tensor\elemwise.py:826: RuntimeWarning: divide by zero encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
c:\Users\Johnny Lee\anaconda3\envs\py39win64\lib\site-packages\theano\tensor\elemwise.py:826: RuntimeWarning: invalid value encountered in multiply
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Sequential sampling (1 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [xi]
>NUTS: [beta, alpha]


Sampling 1 chain for 1_000 tune and 2_000 draw iterations (1_000 + 2_000 draws total) took 10664 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [13]:
results = pd.DataFrame({'var': np.arange(300), 
                        'inclusion_probability':np.apply_along_axis(np.mean, 0, trace['xi']),
                       'beta':np.apply_along_axis(np.mean, 0, trace['beta']),
                       'beta_given_inclusion': np.apply_along_axis(np.sum, 0, trace['xi']*trace['beta'])
                            /np.apply_along_axis(np.sum, 0, trace['xi'])
                       })

In [8]:
results.sort_values('inclusion_probability', ascending = False).head(20)

,var,inclusion_probability,beta,beta_given_inclusion
127,127,1.0000,1.074057,1.074057
18,18,0.8935,0.649910,0.733500
176,176,0.8650,-0.624158,-0.713840
241,241,0.7255,0.512173,0.730294
16,16,0.6400,-0.421841,-0.643205
74,74,0.5515,-0.340213,-0.611754
199,199,0.3960,-0.234576,-0.568144
66,66,0.3945,0.255440,0.635501
135,135,0.3915,-0.187125,-0.594662
136,136,0.2815,-0.160670,-0.556925


In [9]:
#Scoring test.  Score new data from a single posterior sample
test_beta = trace['beta'][0]
test_inc = trace['xi'][0]
test_score = expit(trace['alpha'][0] + np.dot(X2, test_inc * test_beta))  
test_score

array([0.17482076, 0.22571151, 0.21963124, ..., 0.18084265, 0.18985789,
       0.03487203])

In [10]:
estimate = trace['beta'] * trace['xi'] 
y_hat = np.apply_along_axis(np.mean, 1, expit(trace['alpha'] + np.dot(X2, np.transpose(estimate) )) )

In [11]:
#Sanity checks
np.mean(y_hat), np.sum(results.inclusion_probability/300)

(0.29178200192454595, 0.05087833333333333)

In [14]:
submission  = pd.DataFrame({'id':df2.id, 'target':y_hat})
# submission.to_csv('submission.csv', index = False)
submission

,id,target
0,250,0.482081
1,251,0.223246
2,252,0.317099
3,253,0.604170
4,254,0.568997
...,...,...
19745,19995,0.192739
19746,19996,0.700733
19747,19997,0.313685
19748,19998,0.252241
